# MVD 11. cvičení


## 1. část - Collaborative Filtering (item-item)

Vytvořte si matici podle příkladu v přednášce na item-item přístup na snímku 24. Využijte centered cosine similarity pro výpočet podobností stejně jako v přednášce a vyberte dva nejbližší sousedy. Ověřte, že je výsledek v přednášce správný a implementujte funkci tak, aby bylo možné jednoduše spočítat i libovolné další vyhodnocení. 

In [23]:
import numpy as np

In [24]:
data = np.array([
    [1,0,3,0,0,5,0,0,5,0,4,0],
    [0,0,5,4,0,0,4,0,0,2,1,3],
    [2,4,0,1,2,0,3,0,4,3,5,0],
    [0,2,4,0,5,0,0,4,0,0,2,0],
    [0,0,4,3,4,2,0,0,0,0,2,5],
    [1,0,3,0,3,0,0,2,0,0,4,0]
])

In [130]:
import torch
import sklearn

def center_vector(vector: np.array) -> np.array:
    """Centrování vektoru
    Vstupem je vektor a výstupem je centrováný vektor.
    """
    mask = (vector == 0)
    sum = np.sum(vector, axis=1) / np.count_nonzero(vector, axis=1)
    centered = np.where(mask, 0, vector - sum[:,np.newaxis])
    return centered

def similarity(a: np.array, b: np.array, centered = False) -> np.array:
    """
    Vstupem jsou dva vektory a výstupem je hodnota cosinove podobnosti.
    Přepínač: centered
        Vypočte se Pearson Correlation, tedy centrovanou podobnost
    """
    if centered:
        a = center_vector(a)
        b = center_vector(b)
    #return torch.nn.functional.cosine_similarity(a, b, dim=0)
    return sklearn.metrics.pairwise.cosine_similarity(a, b)
    #return np.dot(a, b.T)/(np.linalg.norm(a)*np.linalg.norm(b))

def collaborative_filtering(matrix : np.array, predict: tuple, k : int = 2):
    user, product = predict
    similarities = similarity(matrix[product,:].reshape(1,-1), matrix, centered = True).squeeze()
    indexes = np.argsort(similarities)[::-1][1:k+1]

    rating = np.sum(similarities[indexes] * matrix[indexes, user]) / np.sum(similarities[indexes])
    return rating


In [131]:
collaborative_filtering(matrix=data, predict=(4,0), k=2)

2.5864068669348175

## Bonus - Content-based Filtering

Stáhněte si Kaggle dataset [Spotify Recommendation system](https://www.kaggle.com/bricevergnou/spotify-recommendation). Z datasetu budete potřebovat:

- data.csv = příznaky k jednotlivým skladbám + příznak liked pro klasifikaci

Úkolem je:

1. Načíst data z csv.
2. Vytvořit train (90 %) a test (10 %) split pomocí knihovny [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).
3. Vytvořit model logistické regrese (vlastní implementace / využití knihovny sklearn nebo jiné).
4. Vyhodnotit data na testovací sadě (např. metoda score u LogisticRegression).

**Skóre pro uznání úlohy musí být vyšší než 89 %.**

Dobrovolné: vytvořit graf predikovaných vs aktuálních hodnot

Získání bodu navíc: Využijte i další algoritmy pro binární klasifikaci.